# Sequências - Aula Prática

## RNNs (Recurrent Neural Networks)

Neste notebook, iremos trabalhar com a abordagem inicial de redes neurais recorrentes (RNNs), implementando do zero uma RNN utilizando PyTorch. Além disso, iremos usar RNNs para para realizar uma classificação de litologia.

- **Importante:** caso esteja rodando esse notebook no ambiente da Tatu, favor executar a próxima célula. Caso contrário, basta ignorar a sua execução.

In [ ]:
data_dir = '/pgeoprj/godeep/dados/l2_datasets/publico/force'

# Configuração do ambiente



In [ ]:
import os

import json
import pandas as pd
from tqdm import tqdm
import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, f1_score, precision_score, recall_score, ConfusionMatrixDisplay, balanced_accuracy_score

import torch
import random

import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import matplotlib

Um estado aleatório fixo (`random_state = 42`) é definido para reprodutibilidade entre experimentos.

A configuração do dispositivo garante o uso da GPU, se disponível, com um fallback para a CPU.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device escolhido:', device)

random_state = 42

Device escolhido: cuda


# Carregamento da base de dados

### Definição da classe Force

A classe `Force` foi projetada para lidar com o carregamento e o pré-processamento inicial de dados de registro de poços para classificação de litologia. O conjunto de dados FORCE vem de uma competição feita em 2020.

Abaixo estão os detalhes de sua implementação:

#### **Inicialização de classe (método `__init__`)**
- Parâmetros:
    - `directory`: Caminho para o diretório de dados.
    - `logs`: Uma lista de registros a serem usados dataset FORCE.
    - `verbose`: Se `True`, imprime detalhes adicionais durante a execução.
- Chaves de litologia: Um dicionário que mapeia códigos numéricos de litologia para seus respectivos nomes.

#### **Carregamento de dados (método `open_data`)**
- Objetivo: Método principal para abrir, mesclar e pré-processar dados de registro de poço.
- Retorna um `pandas.DataFrame` contendo dados de registro de poço, e uma instância `LabelEncoder` para codificar rótulos de litologia em valores numéricos de 0 a num_classes-1.

**Principais etapas:**
1. Carregamento de dados: Lê vários arquivos CSV (`train.csv`, `hidden_test.csv`, `leaderboard_test_features.csv`, etc.) em DataFrames separados.
2. Verificações de consistência de dados: Garante valores correspondentes para as colunas `WELL` e `DEPTH_MD` entre dataframes concatenados.
3. Manipulação de valores ausentes: Preenche valores ausentes na coluna `FORCE_2020_LITHOFACIES_CONFIDENCE` com `NaN`.
4. Concatenação e redefinição de índice:*Mescla vários conjuntos de dados (`train`, `leaderboard_test`, etc.) em um DataFrame consolidado e redefine o índice.
5. Codificação de rótulos: mapeia códigos de litologia para seus nomes usando `lithology_keys` e os codifica como inteiros usando `LabelEncoder`.

**Saída:**
- O conjunto de dados pré-processado (`data`) e o codificador de rótulo (`le`) são retornados para uso subsequente no pipeline de modelagem.


In [ ]:
class Force:
    def __init__(self, directory:str, logs:list[str], verbose:bool=False) -> None:
        """
            Arguments:
            ---------
                - directory (str): Path to the directory where data is
                - logs (list[str] or tuple[str]): Logs used from FORCE data
                - verbose (bool): If True, print progress details. Else, does not print anything.
        """

        self.directory = directory
        self.logs = logs

        self.lithology_keys = {30000: 'Sandstone',
                     65030: 'Sandstone/Shale',
                     65000: 'Shale',
                     80000: 'Marl',
                     74000: 'Dolomite',
                     70000: 'Limestone',
                     70032: 'Chalk',
                     88000: 'Halite',
                     86000: 'Anhydrite',
                     99000: 'Tuff',
                     90000: 'Coal',
                     93000: 'Basement'}

        self.data, self.le = self.open_data()

    def open_data(self) -> tuple[pd.DataFrame, LabelEncoder]:
        """
        Main method to open data.
            Arguments:
            ---------
                -
            Return:
            ---------
                - data (pd.DataFrame): Well log dataset fully configured to be used
                - le (LabelEncoder): Label Encoder used to encode lithology classes to consecutive numbers
        """

        train_data = pd.read_csv( os.path.join(self.directory, 'train.csv'), sep=';' )
        hidden_test = pd.read_csv( os.path.join(self.directory, 'hidden_test.csv'), sep=';' )
        leaderboard_test_features = pd.read_csv( os.path.join(self.directory, 'leaderboard_test_features.csv'), sep=';' )
        leaderboard_test_target = pd.read_csv( os.path.join(self.directory, 'leaderboard_test_target.csv'), sep=';' )

        ## A little of consistency checking
        leaderboard_test_target['WELL_tg'] = leaderboard_test_target.WELL
        leaderboard_test_target['DEPTH_MD_tg'] = leaderboard_test_target.DEPTH_MD
        leaderboard_test_target.drop(columns=['WELL', 'DEPTH_MD'], inplace=True)
        leaderboard_test = pd.concat([leaderboard_test_features, leaderboard_test_target], axis=1)

        ## Make sure the values for the WELL and DEPTH_MD columns match between the two concatenated data-frames
        _check_well = np.all( (leaderboard_test.WELL == leaderboard_test.WELL_tg).values )
        _check_depth = np.all( (leaderboard_test.DEPTH_MD == leaderboard_test.DEPTH_MD_tg).values )
        assert _check_well and _check_depth, 'Inconsistency found in leaderboard test data...'

        ## Passed the consistency check, we drop the redundant columns
        leaderboard_test.drop(columns=['WELL_tg', 'DEPTH_MD_tg'], inplace=True)

        ## Note leaderboard_test dataframe does not have the FORCE_2020_LITHOFACIES_CONFIDENCE column. We will therefore fill it with NaNs.
        leaderboard_test['FORCE_2020_LITHOFACIES_CONFIDENCE'] = np.nan

        data = pd.concat([train_data, leaderboard_test, hidden_test], axis=0, ignore_index=True)
        data.sort_values(by=['WELL', 'DEPTH_MD'], inplace=True)
        data.reset_index(drop=True, inplace=True)

        data['LITHOLOGY_NAMES'] = data.FORCE_2020_LITHOFACIES_LITHOLOGY.map(self.lithology_keys)
        data = data[data["LITHOLOGY_NAMES"] != 'Basement']
        le = LabelEncoder()
        data['LITHOLOGY'] = le.fit_transform(data['FORCE_2020_LITHOFACIES_LITHOLOGY'])

        return data, le

# Pré-processamento

## Winsorização de dados de registro de poços (função `remove_quartiles`)
Esta função remove outliers em dados de registro de poços cortando valores com base em quantis superiores e inferiores.

#### Saída
- Um DataFrame (`data`) com outliers removidos para os registros especificados.

In [ ]:
def remove_quartiles(original_data:pd.DataFrame, logs:list[str], q:list=[0.01, 0.99], verbose:bool=True) -> pd.DataFrame:
    """
    Function to apply winsorization (remove outliers by clipping extreme quartiles. Upper or lower quartiles)
        Arguments:
        ---------
            - original_data (pd.DataFrame): Well log data, including lithology column
            - logs (list[str]): List of log names used. Ex: GR, NPHI, ...
            - class_col (str): Name of the lithology column
        Return:
        ---------
            - data (pd.DataFrame): Well log data without outliers.
    """

    data = original_data.copy()
    num_cols = len(logs)

    for i, col in enumerate(logs):
        if verbose:
            print(f'Handling log {i + 1}/{num_cols} - {col}')
        array_data = data[col].values
        only_nans = np.all( np.isnan(array_data) )

        if not only_nans:
            min_quart, max_quart = np.nanquantile(array_data, q=q)

            if verbose:
                print(f'{col}: min: {min_quart:.4f} - max: {max_quart:.4f} ')

            # Set outlier values as nan
            outlier_idx = np.logical_or(array_data < min_quart, array_data > max_quart)
            if verbose:
                print(f'Ignoring {np.sum(outlier_idx)} values')

            # Set series in dataframe with clipped values
            data[col] = data[col].clip(min_quart, max_quart)

    if verbose:
        print()

    return data

## Abrir e pré-processar dados de registro de poço (função `open_and_preprocess_data`)
Esta função carrega, pré-processa e divide dados de registro de poço para treinamento, validação e teste.

#### Processo
1. Carregamento de dados: Usa a classe `Force` para carregar e pré-processar dados.
2. Winsorização: Remove outliers dos registros de poço usando `remove_quartiles`.
3. Divisão de poço: Divide poços em conjuntos de treinamento, validação e teste com base em tamanhos especificados.

#### Saída
- `data`: DataFrame pré-processado.
- `le`: Codificador de rótulo para classes de litologia.
- `well_names`: Lista de todos os nomes de poço.
- `train_wells`, `val_wells`, `test_wells`: Listas de poços para treinamento, validação e teste.


In [ ]:
def open_and_preprocess_data(data_dir:str, logs:list[str], class_col:str, test_size:float, val_size:float, shuffle:bool, random_state:int|np.random.RandomState, verbose:bool=True) -> tuple[pd.DataFrame, LabelEncoder, list, list, list]:

    """
    Function that receives all necessary parameters to open and preprocess data and calls all necessary functions, classes and methods.
        Arguments:
        ---------
            - data_dir (str): Path for folder containing dataset.
            - logs (str): List of names of logs used.
            - class_col (str): Name of the label column (usually 'Lithology')
            - test_size (float): Size of test set. Range: 0-1.
            - val_size (float): Size of validation set. Range: 0-1.
            - shuffle (bool): Wether to shuffle or not while data splitting.
            - random_state (int or np.random.RandomState): Random state to define random operations.
            - verbose (bool): If True, print progress details. Else, does not print anything.
        Return:
        ---------
            - data (pd.DataFrame): Well log dataset fully configured to be used
            - le (LabelEncoder): Label Encoder used to encode lithology classes to consecutive numbers
            - well_names (list[str]): List of all well names contained in dataset
            - train_wells (list[str]): List of train wells after splitting
            - val_wells (list[str] or None): List of validation wells after splitting. Can be None if there is no validation split.
            - test_wells (list[str] or None): List of test wells after splitting. Can be None if there is no test split.
    """

    force_dataset = Force(data_dir, logs)
    data, le = force_dataset.data, force_dataset.le

    data = remove_quartiles(data, logs, verbose=verbose)

    well_names = list(data['WELL'].unique())
    train_wells, test_wells = train_test_split(well_names, test_size=test_size, shuffle=shuffle, random_state=random_state)
    train_wells, val_wells = train_test_split(train_wells, test_size=val_size, shuffle=shuffle, random_state=random_state)


    return data, le, well_names, train_wells, val_wells, test_wells

## Pipeline de pré-processamento de dados
Esta seção usa a função `open_and_preprocess_data` para carregar e preparar dados para o aprendizado de máquina:
1. Carrega o conjunto de dados com remoção de outliers habilitada.
2. Divide os dados em conjuntos de treinamento, validação e teste com base em nomes de poços.
3. Atribui dados de entrada (`X`) e rótulos de destino (`y`) para cada divisão.
4. Normaliza valores de log de poços usando `StandardScaler`.

#### Saída de dados
- `X_train`, `y_train`: Dados de treinamento e rótulos.
- `X_val`, `y_val`: Dados de validação e rótulos.
- `X_test`, `y_test`: Dados de teste e rótulos.

Este pipeline garante pré-processamento consistente e prepara o conjunto de dados para modelagem.

In [ ]:
class Config:
    seq_size = 256
    batch_size = 64

    split_form = 'train_val_test' ## kfold, train_test or train_val_test
    n_splits = 5
    test_size = 0.2
    val_size = 0.1
    shuffle = True

    scaling_method = 'standard' # standard or min-max

    data_dir = data_dir
    logs = ['GR', 'RHOB', 'NPHI', 'DTC']
    logs_info = logs + ['WELL', 'DEPTH_MD', 'LITHOLOGY']
    class_col = 'LITHOLOGY'

    num_classes = 11

    verbose = True

cfg = Config()

In [ ]:
data, le_data, well_names, train_wells, val_wells, test_wells = open_and_preprocess_data(cfg.data_dir, cfg.logs, cfg.class_col,
                                                                                   cfg.test_size, cfg.val_size, cfg.shuffle,
                                                                                   random_state, verbose=cfg.verbose)

Handling log 1/4 - GR
GR: min: 8.9536 - max: 180.3104 
Ignoring 28592 values
Handling log 2/4 - RHOB
RHOB: min: 1.6196 - max: 2.6975 
Ignoring 24838 values
Handling log 3/4 - NPHI
NPHI: min: 0.0491 - max: 0.6245 
Ignoring 19320 values
Handling log 4/4 - DTC
DTC: min: 60.0862 - max: 173.0303 
Ignoring 26878 values



In [ ]:
train_data = data[data['WELL'].isin(train_wells)]
X_train = train_data[cfg.logs_info]
y_train = train_data[cfg.class_col]

val_data = data[data['WELL'].isin(val_wells)]
X_val = val_data[cfg.logs_info]
y_val = val_data[cfg.class_col]

test_data = data[data['WELL'].isin(test_wells)]
X_test = test_data[cfg.logs_info]
y_test = test_data[cfg.class_col]

In [ ]:
scaler = StandardScaler()

X_train[cfg.logs] = scaler.fit_transform(X_train[cfg.logs])
X_val[cfg.logs] = scaler.transform(X_val[cfg.logs])
X_test[cfg.logs] = scaler.transform(X_test[cfg.logs])

<ipython-input-12-36ac85b6a544>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[cfg.logs] = scaler.fit_transform(X_train[cfg.logs])
<ipython-input-12-36ac85b6a544>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[cfg.logs] = scaler.transform(X_val[cfg.logs])
<ipython-input-12-36ac85b6a544>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

# Dataset e DataLoader

### Classe LithologyDataset
Esta classe é um conjunto de dados PyTorch personalizado projetado para classificação de litologia. Ele processa dados de registro de poços, gera sequências e prepara os dados para treinamento do modelo.

#### Criando sequências (método `_create_dataset`)
- Objetivo: Gera sequências de comprimento fixo (`seq_size`) a partir dos dados para cada poço.
- Processo:
1. Itera por cada poço no conjunto de dados.
2. Extrai sequências consecutivas de tamanho `seq_size`.
3. Lida com valores ausentes pulando sequências com NaNs.
4. Acrescenta sequências válidas junto com seus rótulos à lista de sequências.
- Saída: Uma lista de sequências sem valores ausentes.

#### Tamanho do conjunto de dados (método `__len__`)
- Objetivo: Retorna o número total de sequências no conjunto de dados.

#### Obter item (método `__getitem__`)
- Objetivo: Recupera uma única sequência e seus rótulos correspondentes com base no índice.

### Funcionalidade geral
A classe `LithologyDataset`:
1. Prepara dados de registro de poços para classificação de litologia.
2. Lida com geração de sequência, valores ausentes e formatação.
3. Fornece estruturas de dados compatíveis com PyTorch para treinamento de modelos de aprendizado de máquina.


In [ ]:
class LithologyDataset(Dataset):
    def __init__(self, df:pd.DataFrame, labels:pd.Series, logs:list[str], num_classes:int, seq_size:int=100, interval_size:int=100, well_name_column:str='WELL', lithology_column:str='LITHOLOGY') -> None:
        """
            Arguments:
            ---------
                - df (pd.DataFrame): Well log data
                - labels (pd.Series): Column containing lithology classes for each depth
                - logs (list[str]): List of logs used. Ex: GR, NPHI, ...
                - num_classes (int): Number of lithology classes
                - seq_size (int): Size of sequence sent to the model
                - interval_size (int): Size of the interval used to extract consecutive sequences
                - well_name_column (str): Name of the column that indicates the well name in the data
                - lithology_column (str): Name of the lithology column
            Return:
            ---------
                None
        """

        self.data = df
        self.list_of_wells = list(df[well_name_column].unique())
        self.labels = labels
        self.logs = logs
        self.num_classes = num_classes
        self.seq_size = seq_size
        self.interval_size = interval_size
        self.well_name_column = well_name_column
        self.lithology_column = lithology_column
        self.no_missing_logs = self.logs + [self.lithology_column]

        self.data['labels'] = labels
        self.list_of_sequences = self.__create_dataset(self.data, verbose=False)

    def __create_dataset(self, df:pd.DataFrame, verbose:bool=False) -> list:
        """
            Arguments:
            ---------
                - df (pd.DataFrame): Well log data
            Return:
            ---------
                - list_of_sequences (list): list of all sequences without null values in the dataset
        """

        list_of_sequences = list()

        for wellname in tqdm(self.list_of_wells, disable=(not verbose)):

            well_df = df[df[self.well_name_column] == wellname]

            j=0
            while j < well_df.shape[0]-(self.seq_size-1): # Enquanto for possível pegar uma sequência de tamanho seq_size no meu poço

                sequence = well_df.iloc[j:j+self.seq_size]

                # Busca indíces de valores nulos dentro da sequência
                idx_null = [k for k,x in enumerate(sequence[self.no_missing_logs].values) if np.isnan(x).any()]

                # Se não tiver valor nulo na sequência
                if idx_null == []:
                    list_of_sequences.append([wellname, sequence[self.logs], sequence['labels']])
                    j = j + self.interval_size
                # Se tiver, pular para o indíce seguinte ao último valor nulo na sequência
                else:
                    j = j + idx_null[-1] + 1

        return list_of_sequences

    def __len__(self):

        return len(self.list_of_sequences)

    def __getitem__(self, idx) -> tuple[str, torch.Tensor, torch.Tensor]:
        """
            Arguments:
            ---------
                - idx (int): Index for selecting a sample from the dataset
            Return:
            ---------
                - wellname (str): Name of the well from which the sequence is taken
                - well_data_torch (torch.Tensor): Well log sequence
                - labels_torch (torch.Tensor): One-hot-encoded lithology labels sequence
        """

        wellname, sequence, labels = self.list_of_sequences[idx]
        # To numpy
        sequence_numpy = sequence.to_numpy()
        sequence_numpy = np.reshape(sequence_numpy, (-1, len(self.logs)))

        # Create one-hot vector to represent labels
        labels_numpy = np.array([np.array([1. if i==label else 0. for i in range(self.num_classes)]) for label in labels.to_numpy()])

        # To torch
        well_data_torch = torch.from_numpy(sequence_numpy).float()
        labels_torch = torch.from_numpy(labels_numpy).float()

        return wellname, well_data_torch, labels_torch

### Criando Datasets e DataLoaders

Esta seção prepara os dados para treinamento, validação e teste utilizando a classe `LithologyDataset` e o `DataLoader` do PyTorch. Os DataLoaders facilitam o processamento em lote durante o treinamento e a avaliação do modelo.

#### Datasets
**Dataset de treinamento, validação ou teste (`{split_type}_dataset`)**:
- Entradas: `X_{split_type}`, `y_{split_type}` e parâmetros de configuração.
- As sequências são geradas usando a classe `LithologyDataset` com tamanho de sequência especificado (`cfg.seq_size`) e tamanho de intervalo.

#### DataLoaders
**DataLoader (`{split_type}_dataloader`)**:
- Envolve o `{split_type}_dataset` para processamento em lote eficiente.
- Tamanho do lote: `cfg.batch_size`.
- O embaralhamento habilitado (`shuffle=True`) garante a randomização durante o treinamento.

### Objetivo
Esta configuração garante:
1. Carregamento de dados eficiente e escalável para treinamento, validação e teste.
2. Compatibilidade com o loop de treinamento do PyTorch.
3. Manipulação adequada de sequências com tamanhos de lote configuráveis e embaralhamento de dados conforme necessário.


In [ ]:
train_dataset = LithologyDataset(X_train, y_train, cfg.logs, cfg.num_classes, seq_size=cfg.seq_size, interval_size=cfg.seq_size)
train_dataloader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)

val_dataset = LithologyDataset(X_val, y_val, cfg.logs, cfg.num_classes, seq_size=cfg.seq_size, interval_size=cfg.seq_size)
val_dataloader = DataLoader(val_dataset, batch_size=cfg.batch_size, shuffle=False)

test_dataset = LithologyDataset(X_test, y_test, cfg.logs, cfg.num_classes, seq_size=cfg.seq_size, interval_size=cfg.seq_size)
test_dataloader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False)

<ipython-input-13-33781b61fbc3>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['labels'] = labels
<ipython-input-13-33781b61fbc3>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['labels'] = labels
<ipython-input-13-33781b61fbc3>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

# Modelos

#### **Classe RNN**

1. Implemente uma RNN. Ela deve ser projetada para manipular dados sequenciais, e ter uma camada totalmente conectada para gerar saídas. Deve ser possível configurar seu modelo para que seja unidirecional ou bidirecional.

In [ ]:
# Implemente a sua solução aqui

# Treinamento

### Função de avaliação (`evaluate`)

2. Implemente uma função para avaliar o desempenho de um modelo. Calcule acurácia, MCC, precisão, recall e F1-score. Crie uma matriz de confusão com rótulos de classe de litologia.

In [ ]:
# Implemente a sua solução aqui

### Configuração do modelo, função de perda e otimizador

3. Configure e instancie dois modelos: RNN e BiRNN. Depois, defina uma função de perda e um otimizador para o treinamento. **Dica:** Não se esqueça de enviar os modelos para a GPU.


In [ ]:
# Implemente a sua solução aqui

### Loop de treinamento

4. Implemente a função `train` e treine cada um dos modelos.

In [ ]:
def train(model, epochs=100):
    
    # Implemente a sua solução aqui

### Avaliação final

5. Avalie cada um dos modelos por meio da função de avaliação.

In [ ]:
# Implemente a sua solução aqui